In [ ]:
import sys
sys.path.append('C:\sesac_lagchain\project3\libs')  # libs 폴더가 위치한 경로 추가


from libs import models
from libs import messages
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

## 로컬 이미지 가져와서 ocr

In [ ]:

import cv2
import numpy as np
import pytesseract

# Tesseract 실행 파일 경로 설정 (Windows)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# 이미지 불러오기
image_path = r"C:\sesac_lagchain\project3\images\1445.jpg"
image = cv2.imread(image_path)

# 1️⃣ **그레이스케일 변환 (색상 제거)**
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 2️⃣ **샤프닝 적용 (흐릿한 글씨 선명하게)**
sharpen = cv2.GaussianBlur(gray, (0, 0), 3)
sharpen = cv2.addWeighted(gray, 1.5, sharpen, -0.5, 0)

# 3️⃣ **노이즈 제거 (가우시안 블러 적용)**
blur = cv2.GaussianBlur(sharpen, (3, 3), 0)

# 4️⃣ **이진화 처리 (Threshold 적용 - 배경 제거)**
_, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 5️⃣ **기울어진 텍스트 정렬 (회전 보정)**
coords = np.column_stack(np.where(binary > 0))
angle = cv2.minAreaRect(coords)[-1]
if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
(h, w) = binary.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
binary = cv2.warpAffine(binary, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

# 6️⃣ **OCR 실행 (한국어 + 영어)**
custom_config = r'--oem 3 --psm 3'  # 문서 전체 분석 모드
text = pytesseract.image_to_string(binary, lang="kor+eng", config=custom_config)

# 결과 출력
print("📝 OCR 결과:\n", text)

print(text)


## 인터넷상 이미지 주소 가져와서 ocr

In [3]:
import cv2
import numpy as np
import pytesseract
import requests
from io import BytesIO

# Tesseract 실행 파일 경로 설정 (Windows)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# **1️⃣ URL에서 이미지 다운로드**
image_url = r"http://sesac.seoul.kr/common/crosseditor/binary/images/000011/2025년_서울청년정책네트워크_모집_정방형_포스터_정방형_포스터_(1).jpg"
response = requests.get(image_url)

if response.status_code == 200:
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
else:
    print(f"❌ 이미지 다운로드 실패: {response.status_code}")
    exit()

# **2️⃣ 그레이스케일 변환 (색상 제거)**
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# **3️⃣ 샤프닝 적용 (흐릿한 글씨 선명하게)**
sharpen = cv2.GaussianBlur(gray, (0, 0), 3)
sharpen = cv2.addWeighted(gray, 1.5, sharpen, -0.5, 0)

# **4️⃣ 노이즈 제거 (가우시안 블러 적용)**
blur = cv2.GaussianBlur(sharpen, (3, 3), 0)

# **5️⃣ 이진화 처리 (Threshold 적용 - 배경 제거)**
_, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# **6️⃣ OCR 실행 (한국어 + 영어)**
custom_config = r'--oem 3 --psm 6'  # 한 줄 텍스트 인식 모드
text = pytesseract.image_to_string(binary, lang="kor+eng", config=custom_config)

# **7️⃣ 결과 출력**
print("📝 OCR 결과:\n", text)


📝 OCR 결과:
 Sis 서 울 청 년 징 책 넷 트 워 크 "얗얽』꼬
ㅡ 빼 o
2025 N=zBUusHueaS
타 2
— BOR 모
ieitoks ㅡ | — |
22 EH
2024.276.( 월 ) 07° esenaeser cavemen ane um
~ 2025.01 12.(2) 24:00 ueeavinaeue a 위 한 의 제 토 른 및 사 회 기 여 살 등
넨챔브쿰 BHO toto
~ 신 청 하 1 - 서 울 청 년 점 책 네 트 워 크

서 롤 시 거 주 히 거 나 1200
생 활 권 으 로 하 는 022130-8 세 지 8
만 19~39 세 첨 년 즐 쨈

+ Pro

rw ad Se *

떼 * ps

\ -. | | /



### 인터넷상 이미지 주소 채택 -> for 문

In [ ]:
import cv2
import numpy as np
import pytesseract
import requests
import json

# Tesseract 실행 파일 경로 설정 (Windows)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# **1️⃣ JSON 파일 로드**
json_file_path = "offline_courses_merged_image.json"

with open(json_file_path, "r", encoding="utf-8") as f:
    image_data = json.load(f)

# **2️⃣ URL에서 이미지 가져와 OCR 실행 함수**
def get_ocr_text_from_url(image_url):
    """URL에서 이미지를 불러와 OCR을 실행하고 텍스트를 반환"""
    try:
        response = requests.get(image_url, timeout=10)
        if response.status_code != 200:
            print(f"❌ 이미지 다운로드 실패: {image_url}")
            return ""

        image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

        if image is None:
            print(f"❌ 이미지 로드 실패: {image_url}")
            return ""

        # **이미지 전처리**
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (3, 3), 0)
        _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # **OCR 실행**
        custom_config = r'--oem 3 --psm 6'  
        text = pytesseract.image_to_string(binary, lang="kor+eng", config=custom_config)

        return text.strip()  # 앞뒤 공백 제거 후 반환

    except Exception as e:
        print(f"❌ OCR 처리 중 오류 발생 ({image_url}): {e}")
        return ""

# **3️⃣ OCR 실행 및 결과 저장**
ocr_results = {}

for key, value in image_data.items():
    if value == "None":
        ocr_results[key] = ""  # OCR 실행 안 함
    elif isinstance(value, str):  # 단일 URL
        ocr_results[key] = get_ocr_text_from_url(value)
    elif isinstance(value, list):  # 여러 개의 URL
        combined_text = "\n".join([get_ocr_text_from_url(url) for url in value])
        ocr_results[key] = combined_text
    print(key,'완료!')


In [ ]:

# **4️⃣ 결과 저장**
output_file = "ocr_results.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(ocr_results, f, ensure_ascii=False, indent=4)

print(f"✅ OCR 결과 저장 완료: {output_file}")


In [ ]:
print(ocr_results)

## orc to text

In [ ]:
import openai
from dotenv import load_dotenv
import os
import json

def chunk_text(text, max_length=1000):
    """
    긴 텍스트를 max_length 크기의 청크로 분할하는 함수
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        if current_length + len(word) + 1 > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(word)
        current_length += len(word) + 1
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

# .env 파일 로드
load_dotenv('C:\sesac_lagchain\.env')

# 환경 변수에서 OpenAI API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

# 시스템과 사용자 프롬프트 설정
system_prompt = """
당신은 OCR로 변환된 문서를 사람이 읽기 쉽게 정제하는 역할을 합니다.  
다음의 원칙을 따릅니다.  

1. **띄어쓰기 및 문장 구조 수정**  
   - 단어를 추가하거나 수정하지 않고, 띄어쓰기만 자연스럽게 조정하여 원본 텍스트를 최대한 유지합니다. 
   - 문장이 어색하거나 잘못된 문법이 있으면 올바르게 수정합니다.  

2. **가독성 향상**  
   - 문장을 논리적으로 정리하여 가독성을 높이고, 목차가 있다면 구분하여 표시하세요. 단, 주어진 문서의 단어 외의 단어는 절대 추가하지 마세요.   

3. **목차 및 섹션 정리**  
   - 제목과 본문을 명확하게 구분합니다.  
   - 관련된 내용을 묶어서 단락을 구성합니다.  

4. **불필요한 기호 및 잘못된 OCR 인식 수정**  
   - 의미 없는 특수문자, 깨진 문자(예: @, #, %) 등을 제거합니다.  
   - 잘못 인식된 단어를 원래 의미에 맞게 수정합니다.
"""
user_prompt = """
다음은 OCR을 통해 변환된 문서입니다.  
- 사람이 쉽게 읽고 이해할 수 있도록 띄어쓰기와 문장 구조를 수정하세요.  
- 중복되거나 불필요한 단어는 제거하세요.  
- 문단을 적절히 나누고, 논리적인 흐름을 유지하세요.  
- 제목과 본문을 구분하여 명확한 문서 구조를 만들어 주세요.
- 의미 없는 특수문자, 깨진 문자(예: @, #, %) 등을 제거합니다.
- OCR로 변환된 문서에서 참조되지 않은 문장은 삭제하세요
"""

def analyze_text(extracted_text):
    """
    텍스트를 청크 단위로 나누어 OpenAI API에 전달 후 결과를 합치는 함수
    """
    chunks = chunk_text(extracted_text)
    results = []
    
    for chunk in chunks:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
                {"role": "user", "content": chunk}
            ],
            max_tokens=500,
            temperature=0.7
        )
        results.append(response.choices[0].message.content)
    
    return "\n".join(results)

### 메인 ###

# with open(r'C:\sesac_lagchain\project3\ocr_results.json', "r", encoding="utf-8") as file:
#     ocr_dict = json.load(file)


ocr2text = {}
for key, ocr_text in ocr_dict.items():
    ocr2text[key] = analyze_text(ocr_text)
    print(key,'완료')


2795 완료
2796 완료
2797 완료
2798 완료
2805 완료
2806 완료
2810 완료
2812 완료
2813 완료
2814 완료
2818 완료
2820 완료
2821 완료
2823 완료
2824 완료
2825 완료
2826 완료
2827 완료
2828 완료
2829 완료
2831 완료
2833 완료
2834 완료
2835 완료
2836 완료
2838 완료
2839 완료
2840 완료
2841 완료
2843 완료
2844 완료
2845 완료
2846 완료
2848 완료
2850 완료
2851 완료
2852 완료
2853 완료
2856 완료
2857 완료
2858 완료


In [10]:
# **4️⃣ 결과 저장**
ocr2text_filename = "ocr2text_result2.json"
with open(ocr2text_filename, "w", encoding="utf-8") as f:
    json.dump(ocr2text, f, ensure_ascii=False, indent=4)

print(f"✅ OCR 결과 저장 완료: {ocr2text}")

✅ OCR 결과 저장 완료: {'2795': '**1. MAKSeSAC)SS BNA SW 21 교육생 모집**\n\n- 모집 기간: 2024. 8.21(수) ~ 10.03(목), 마감일: 10/17\n- 2차 모집: 2024.10.04(금) ~ 10.10(목)\n- 추가 모집: 2024.10.11(금) ~ 10.17(목)\n\n※ 상기 일정은 추진 현황에 따라 일부 변경될 수 있음.\n\n- 모집 대상: 30명 \n\n시니어, 청년, 구직자(장애자료확인 필요), 숙박사업자 등이 대상입니다.\n\n- 교육 기간: 2024.10.23 ~ 2025.2.20(4개월)\n\n**2. 2024년 청년취업사관학교 디지털 교육과정**\n\n국내 1위 교육 출판 기업 천재교육과 함께합니다. 2024년 국내 에듀테크 시장 예상 성장률, 2025년 약 10조 원. 2025년에는 디지털 교과서 도입으로 에듀테크 산업의 성장이 예상됩니다.\n\n에듀테크 플랫폼에서는 학습 데이터 분석을 통해 학습자의 약점을 파악하고, 이를 보완할 맞춤형 학습경로를 제시합니다. 또한, 자동화된 관리 기능을 통해 교육자들이 학습 관리에 소요되는 시간을 절감할 수 있습니다. \n\n에듀테크에 통합된 학습지가 언제\n**미래 교육의 혁신**  \n"올 경험하고 싶다면, 천재교육과 함께하세요! 비전공자도 쉽게 접근할 수 있도록 기초 프로그래밍부터 시작하여 고급 AI AHS 개발 기술까지 체계적으로 학습할 수 있습니다."\n\n**교육 내용**  \n1. 기초부터 심화까지 체계적인 학습\n   - 기계 학습 및 데이터 전처리 역량 강화\n   - 자연어 처리 및 제너레이티브 시기술 습득\n   - 대규모 언어 모델 활용 및 프롬프팅 기술 이해\n2. AWS 클라우드 기초, 인프라 관리 기술 습득\n   - API 설계 및 구현\n   - B/eh 개발 역량 강화\n   - Ae 개발, 배포 및 유지 보수 기술 습득\n3. 프로젝트 기반 학습으로 실무능력 강화\n\n"성장할 수 있습니다!"\n\n**교육 특장점*